In [1]:
#!pip install -r requirements.txt

In [2]:
import yaml
import timeit
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from utility import *
from ultralytics import settings
from registry import get_model, get_pipeline
#settings.update({'datasets_dir': os.path.abspath('.')})


Install mish-cuda to speed up training and inference. More importantly, replace the naive Mish with MishCuda will give a ~1.5G memory saving during training.


[path_manager] + C:\Users\user\Desktop\Aerial Object Detection\AERIAL_OBJECT_DETECTION\Models\YOLOH
 👀  현재 sys.path[0] : C:\Users\user\Desktop\Aerial Object Detection\AERIAL_OBJECT_DETECTION\Models\YOLOH
 📂  root 내용: ['.gitignore', 'benchmark.py', 'config', 'data', 'demo.py', 'eval.py', 'evaluator', 'models', 'README.md', 'test.py']
📂  root/models/yoloh : True


c:\Users\user\anaconda3\envs\AerialObjectDetection\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\user\anaconda3\envs\AerialObjectDetection\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
C:\Users\user\Desktop\Aerial Object Detection\AERIAL_OBJECT_DETECTION\Models\YOLOH\models\backbone\convnext.py:204: UserWarning: Overwriting convnext_tiny in registry with models.backbone.convnext.convnext_tiny. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def convnext_tiny(pretrained=False, res_dilation=False, **kwargs):
C:\U

[path_manager] - C:\Users\user\Desktop\Aerial Object Detection\AERIAL_OBJECT_DETECTION\Models\YOLOH


C:\Users\user\Desktop\Aerial Object Detection\AERIAL_OBJECT_DETECTION\Models\YOLOH\models\backbone\convnext.py:204: UserWarning: Overwriting convnext_tiny in registry with models.backbone.convnext.convnext_tiny. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def convnext_tiny(pretrained=False, res_dilation=False, **kwargs):
C:\Users\user\Desktop\Aerial Object Detection\AERIAL_OBJECT_DETECTION\Models\YOLOH\models\backbone\convnext.py:211: UserWarning: Overwriting convnext_small in registry with models.backbone.convnext.convnext_small. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  def convnext_small(pretrained=False, res_dilation=False, **kwargs):
C:\Users\user\Desktop\Aerial Object Detection\AERIAL_OBJECT_DETECTION\Models\YOLOH\models\backbone\convnext.py:218: UserWarning: Overwriting convnext_base in registry with models.backbone.convnext.convnext_bas

In [3]:
project_name = 'Object Detection'
# dataset_root 가 상대경로일 경우 colab 환경에서는 무조건 앞에 datasets 를 붙여서 찾는 문제가 있음
dataset_root = 'Datasets'
output_dir = 'output'
dataset_names = ['COCO_airplane']
model_names = ['YoloOW', 'yoloh18', 'yoloh50', 'yoloh101' ]
iterations = [1, 5]
epochs = 20
early_stop = 20
batch_size = 16
image_size = 640
optimizer = 'AdamW'
lr = 1e-3
momentum = 0.9
weight_decay = 1e-4
optim_args = {'optimizer': optimizer, 'lr': lr, 'momentum': momentum, 'weight_decay': weight_decay}
devices = [0]
device = torch.device("cuda:"+str(devices[0])) if len(devices)>0 else torch.device("cpu")
train_split = 0.6
val_split = 0.2
test_split = 0.2
num_workers = 2

In [ ]:
from utility.utils import control_random_seed, update_dataset_paths, format_measures, merge_and_update_df


Experiments_Time = datetime.now().strftime("%y%m%d_%H%M%S")
print('Experiment Start Time:',Experiments_Time)

ex_dict = {}
ex_dict['Experiment Time'] = Experiments_Time;ex_dict['Epochs'] = epochs;
ex_dict['Batch Size'] = batch_size;
ex_dict['Early Stop'] = early_stop;
ex_dict['Device'] = device
ex_dict['Optimizer'] = optimizer;
ex_dict['LR']=optim_args['lr']; ex_dict['Weight Decay']=optim_args['weight_decay'];ex_dict['Momentum']=optim_args['momentum'];
#ex_dict['Hyp'] = dict(lr0=ex_dict['LR'], momentum=ex_dict['Momentum'], weight_decay=ex_dict['Weight Decay'])
ex_dict['Image Size'] = image_size
ex_dict['Output Dir'] = output_dir
ex_dict['Num Workers'] = num_workers

for iteration in range(iterations[0], iterations[1]+1):
    print(f'(Iter {iteration})')
    seed = iteration
    ex_dict['Iteration'] = iteration
    for j, Dataset_Name in enumerate(dataset_names):
        print(f'Dataset: {Dataset_Name} ({j+1}/{len(dataset_names)})');
        control_random_seed(seed)
        data_yaml_path = f"{dataset_root}/{Dataset_Name}/data_iter_{iteration:02d}.yaml"
        with open(data_yaml_path, 'r') as f:
            data_config = yaml.load(f, Loader=yaml.FullLoader)
        ex_dict['Dataset Name'] = Dataset_Name; ex_dict['Data Config'] = data_yaml_path; ex_dict['Number of Classes'] = data_config['nc']; ex_dict['Class Names'] = data_config['names'];
        ex_dict['Split Size'] = f"{data_config['train_count']}/{data_config['val_count']}/{data_config['test_count']}"
        update_dataset_paths(dataset_root, Dataset_Name, iteration)
        
        for k, model_name in enumerate(model_names):
            print(f'{model_name} ({k+1}/{len(model_names)}) (Iter {iteration})', end=' ')
            print(f'Dataset: {Dataset_Name} ({j+1}/{len(dataset_names)})', end=' ')
            control_random_seed(seed)
            
            model = get_model(model_name, ex_dict)
            train_fn, eval_fn = get_pipeline(model_name)

            ex_dict['Model Name'] = model_name; ex_dict['Model']=model;
            start = timeit.default_timer()

            ex_dict = train_fn(ex_dict)
            ex_dict = eval_fn(ex_dict)

            ex_dict['Train-Test Time'] = timeit.default_timer() - start
            #################
            if ex_dict.get('Test Results') is None:
                raise RuntimeError("평가 결과가 없습니다. 위의 체크리스트를 참고해 원인부터 해결하세요.")

            eval_dict = format_measures(ex_dict.get('Test Results'))
            output_csv = f"{ex_dict['Experiment Time']}_{project_name}_Results.csv"
            merge_and_update_df(ex_dict, eval_dict, output_csv, exclude_columns=['Model', 'Train Results', 'Test Results'])

Experiment Start Time: 250424_213126
(Iter 1)
Dataset: COCO_airplane (1/1)
YoloOW (1/4) (Iter 1) Dataset: COCO_airplane (1/1) [YoloOW] Using config → C:\Users\user\Desktop\Aerial Object Detection\AERIAL_OBJECT_DETECTION\Models\YoloOW\cfg\training\yoloOW.yaml

[trainer] 🔍 Checking anchor fit to dataset...
[trainer] ⚠️ model has no `.anchors` attribute. Skipping anchor check.
Optimizer: adamw
--momentum: 0.9
--weight_decay: 0.0001


[YoloOW] Epoch 1/20:   0%|          | 0/41 [00:00<?, ?it/s]c:\Users\user\Desktop\Aerial Object Detection\AERIAL_OBJECT_DETECTION\utility\trainer.py:92: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\Scalar.cpp:23.)
  total_loss += loss.item()
[YoloOW] Epoch 1/20: 100%|██████████| 41/41 [00:35<00:00,  1.16it/s, Loss=2.0080, box=0.116, obj=0.009, cls=0.000, lr=0.000325]



[YoloOW] ✅ Epoch 1/20 complete in 35.46s | Avg Loss: 1.6461

[YoloOW] 📊 Running evaluation after epoch 1 ...
[eval_yoloow_model] ⚠️ No valid detections found. Evaluation aborted.


[YoloOW] Epoch 2/20: 100%|██████████| 41/41 [00:32<00:00,  1.26it/s, Loss=1.3478, box=0.082, obj=0.002, cls=0.000, lr=0.000659]



[YoloOW] ✅ Epoch 2/20 complete in 32.74s | Avg Loss: 1.3804

[YoloOW] 📊 Running evaluation after epoch 2 ...
[eval_yoloow_model] ⚠️ No valid detections found. Evaluation aborted.


...

[YoloOW] Epoch 19/20: 100%|██████████| 41/41 [00:50<00:00,  1.23s/it, Loss=849919.8750, box=0.171, obj=53119.820, cls=0.000, lr=0.000026]



[YoloOW] ✅ Epoch 19/20 complete in 50.70s | Avg Loss: 222293.1923

[YoloOW] 📊 Running evaluation after epoch 19 ...
[eval_yoloow_model] ⚠️ No valid detections found. Evaluation aborted.


[YoloOW] Epoch 20/20: 100%|██████████| 41/41 [00:49<00:00,  1.20s/it, Loss=2.8391, box=0.177, obj=0.000, cls=0.000, lr=0.000007]



[YoloOW] ✅ Epoch 20/20 complete in 49.42s | Avg Loss: 202150.7208

[YoloOW] 📊 Running evaluation after epoch 20 ...
[eval_yoloow_model] ⚠️ No valid detections found. Evaluation aborted.
[eval_yoloow_model] ⚠️ No valid detections found. Evaluation aborted.


AttributeError: 'NoneType' object has no attribute 'box'

In [ ]:
import torch

print(f"CUDA 사용 가능 여부: {torch.cuda.is_available()}")
print(f"CUDA 디바이스 수: {torch.cuda.device_count()}")
print(f"현재 사용 디바이스: {torch.cuda.current_device()}")
print(f"디바이스 이름: {torch.cuda.get_device_name(0)}")
print(f"PyTorch CUDA 버전: {torch.version.cuda}")
print(f"PyTorch 버전: {torch.__version__}")


CUDA 사용 가능 여부: True
CUDA 디바이스 수: 1
현재 사용 디바이스: 0
디바이스 이름: NVIDIA GeForce RTX 5060 Ti
PyTorch CUDA 버전: 12.8
PyTorch 버전: 2.8.0.dev20250420+cu128


cat: /root/.config/Ultralytics/settings.json: No such file or directory
